## Vistual inertia scheduling

In [1]:
import andes
import os

from statistics import fmean
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import gurobipy as gb
import pandas as pd
import numpy as np
import logging

logger = logging.getLogger(__name__)

# from opf import dcopf
from visopf import vis1

## Main - IEEE 39


### load case from andes excel

In [2]:
# get andes case from excel
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee39_vis.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)
# ssa.config.mva = 1000

ssa.config.mva

100

### load norm parameter

In [3]:
# prepare nn data for visopf
data_path = dir_path + '/VIS_opf/NN_train'

fnorm = pd.read_csv(data_path + '/fnorm.csv')
pnorm = pd.read_csv(data_path + '/pnorm.csv')
norm = {'fnorm': fnorm, 'pnorm': pnorm }
norm


{'fnorm':            M         D         Fg          Rg    fnadir
 0  14.988180  7.492874  80.045126  124.959573 -0.013860
 1   2.891501  1.438628  40.350784   43.546600  0.006387,
 'pnorm':            M         D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  14.988180  7.492874  80.045126  124.959573  4.018493  2.004602  0.143188
 1   2.891501  1.438628  40.350784   43.546600  2.317657  1.151364  0.086313}

In [4]:
# example to call norm data
norm['fnorm']['M'].iloc[0] # mean
norm['fnorm']['M'].iloc[1] # std 

2.891501173118185

### load nn parameter

In [5]:
fw1 = pd.read_csv(data_path + '/fw1.csv', header=None)
fw2 = pd.read_csv(data_path + '/fw2.csv', header=None)

fb1 = pd.read_csv(data_path + '/fb1.csv', header=None)
fb2 = pd.read_csv(data_path + '/fb2.csv', header=None)

In [6]:
pw1 = pd.read_csv(data_path + '/pw1.csv', header=None)
pw2 = pd.read_csv(data_path + '/pw2.csv', header=None)

pb1 = pd.read_csv(data_path + '/pb1.csv', header=None)
pb2 = pd.read_csv(data_path + '/pb2.csv', header=None)

In [7]:
nn = {
        'fw1': fw1,
        'fw2': fw2,       
        'fb1': fb1,
        'fb2': fb2,
        'pw1': pw1,
        'pw2': pw2,
        'pb1': pb1,
        'pb2': pb2,
    }

### test opf model

In [8]:
ss = vis1(norm=norm, nn=nn, dpe=0.05)
ss.norm

Restricted license - for non-production use only - expires 2023-10-25


{'fnorm':            M         D         Fg          Rg    fnadir
 0  14.988180  7.492874  80.045126  124.959573 -0.013860
 1   2.891501  1.438628  40.350784   43.546600  0.006387,
 'pnorm':            M         D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  14.988180  7.492874  80.045126  124.959573  4.018493  2.004602  0.143188
 1   2.891501  1.438628  40.350784   43.546600  2.317657  1.151364  0.086313}

In [9]:
ss.norm['fnorm']['M'].iloc[0]

14.988179599999995

In [10]:
ss.nn['fw1']

,0,1,2,3
0,0.007024,-0.009032,-0.003725,0.000279
1,0.011937,-0.004678,0.104979,0.009249
2,-0.021316,0.001189,-0.056554,-0.015670
3,-0.012752,-0.000368,-0.006217,0.004011
4,-0.015508,0.020023,-0.125958,-0.015702
...,...,...,...,...
59,-0.002516,0.007691,0.011266,0.005049
60,-0.036352,0.001531,-0.069906,-0.031887
61,-0.023868,0.009889,-0.104516,-0.015767
62,-0.006000,0.003099,-0.005233,-0.009474


In [11]:
ss.nn['fw1'][1].iloc[1]
# ss.nn['fb1']

-0.004678414

In [12]:
vsg_ieee14 = ['PV_6', 'PV_7']
vsg_ieee39 = ['PV_1', 'PV_6', 'PV_8', 'PV_9']

ss.from_andes(ssa, vsg_ieee39)

ss.gen

,idx,u,name,Sn,Vn,bus,p0,pmax,pmin,v0,...,ramp30,type,p_pre,band,K,M,D,R,Mvsg,Dvsg
0,PV_1,1.0,PV_1,10.400,34.5,30,4.360864,15.0,2.0,1.035534,...,600,2,0,13.0,1,0.00000,0.000,0.000000,104.000,52.000
1,PV_2,1.0,PV_2,8.360,34.5,31,6.460000,9.0,1.5,1.013246,...,600,1,0,7.5,1,67.38160,8.360,0.005981,0.000,0.000
2,PV_3,1.0,PV_3,8.437,21.0,32,7.250000,8.0,1.0,1.020528,...,600,1,0,7.0,1,60.40892,8.437,0.005926,0.000,0.000
3,PV_4,1.0,PV_4,11.748,21.0,33,6.520000,7.0,1.0,1.013430,...,600,1,0,6.0,1,102.44256,11.748,0.004256,0.000,0.000
4,PV_5,1.0,PV_5,10.802,15.5,34,5.200000,7.0,1.0,1.019109,...,600,1,0,6.0,1,56.17040,10.802,0.004629,0.000,0.000
5,PV_6,1.0,PV_6,10.857,15.5,35,6.900000,8.0,1.0,1.060000,...,600,2,0,7.0,1,0.00000,0.000,0.000000,65.142,54.285
6,PV_7,1.0,PV_7,10.252,12.5,36,5.900000,7.0,1.0,1.060000,...,600,1,0,6.0,1,84.88656,10.252,0.004877,0.000,0.000
7,PV_8,1.0,PV_8,9.702,12.5,37,3.300000,7.0,1.0,1.013996,...,600,2,0,6.0,1,0.00000,0.000,0.000000,77.616,29.106
8,PV_9,1.0,PV_9,16.841,34.5,38,7.800000,10.0,1.0,1.052803,...,600,2,0,9.0,1,0.00000,0.000,0.000000,101.046,84.205
9,Slack_10,1.0,Slack_10,11.990,345.0,39,5.741700,15.0,3.0,1.030000,...,600,1,0,12.0,1,119.90000,11.990,0.004170,0.000,0.000


In [13]:
ss.norm

{'fnorm':            M         D         Fg          Rg    fnadir
 0  14.988180  7.492874  80.045126  124.959573 -0.013860
 1   2.891501  1.438628  40.350784   43.546600  0.006387,
 'pnorm':            M         D         Fg          Rg      Mvsg      Dvsg     Ppeak
 0  14.988180  7.492874  80.045126  124.959573  4.018493  2.004602  0.143188
 1   2.891501  1.438628  40.350784   43.546600  2.317657  1.151364  0.086313}

In [14]:
a_test = []
for i in range(64):
    a_test.append('a'+str(i))
# a_test

In [15]:
ss.gendict.keys()

dict_keys(['PV_1', 'PV_2', 'PV_3', 'PV_4', 'PV_5', 'PV_6', 'PV_7', 'PV_8', 'PV_9', 'Slack_10'])

In [16]:
ss.nn['fw2'][4].iloc[0]

-0.13017723

In [17]:
ss.nn['fb1']

,0
0,-0.290703
1,0.202561
2,0.151374
3,-0.195499
4,0.256226
...,...
59,-0.389984
60,0.222762
61,0.228741
62,-0.170625


In [18]:
ss.mva

100

In [19]:
ss.nn['fb2'][0].iloc[0]

0.26405954

In [20]:
ss.build()

# ss.mdl.optimize()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.


In [21]:
ss.get_res()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 69 rows, 38 columns and 546 nonzeros
Model fingerprint: 0xec486ba4
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [4e-01, 6e+01]
Presolve removed 54 rows and 30 columns
Presolve time: 0.00s
Presolved: 15 rows, 14 columns, 120 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.8564000e+01   7.957149e+01   0.000000e+00      0s
      15    5.8564000e+01   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.856400000e+01


Successfully solve vis1.


(        gen         pg  pru  prd
 0      PV_1  12.515789  0.0  0.0
 1      PV_2   5.895637  0.0  0.0
 2      PV_3   8.000000  0.0  0.0
 3      PV_4   5.851096  0.0  0.0
 4      PV_5   7.000000  0.0  0.0
 5      PV_6   4.800000  0.0  0.0
 6      PV_7   3.501478  0.0  0.0
 7      PV_8   7.000000  0.0  0.0
 8      PV_9   1.000000  0.0  0.0
 9  Slack_10   3.000000  0.0  0.0,
     gen  Mvsg  Dvsg
 0  PV_1   0.0   0.0
 1  PV_6   0.0   0.0
 2  PV_8   0.0   0.0
 3  PV_9   0.0   0.0)

In [22]:
ss.load.p0.sum()


58.56399999999999

In [23]:
ss.costdict

{'PV_1': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_2': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_3': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_4': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_5': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_6': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_7': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_8': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'PV_9': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0},
 'Slack_10': {'c2': 0, 'c1': 1, 'c0': 0, 'cr': 0, 'cru': 0, 'crd': 0}}

In [24]:
ss.mdl.display()

Minimize
<gurobi.LinExpr: pg[PV_1] + pg[PV_2] + pg[PV_3] + pg[PV_4] + pg[PV_5] + pg[PV_6]
+ pg[PV_7] + pg[PV_8] + pg[PV_9] + pg[Slack_10]>
Subject To
PowerBalance: <gurobi.LinExpr: pg[PV_1] + pg[PV_2] + pg[PV_3] + pg[PV_4] + pg[PV_5] +
 pg[PV_6] + pg[PV_7] + pg[PV_8] + pg[PV_9] + pg[Slack_10]> = 58.564
Line_1_U: <gurobi.LinExpr: -0.5607822357546095 pg[PV_1] + -0.6089602208400834 pg[PV_2]
+ -0.398309791005728 pg[PV_3] + -0.4219560834463752 pg[PV_4] + -0.5066725368514862
pg[PV_5] + -0.5066725368514862 pg[PV_6] + -0.5066725368514856 pg[PV_7] +
-0.5066725368514856 pg[PV_8] + -0.5988298865678321 pg[PV_9] + -0.5607822357546095
 pg[Slack_10]> <= -20.9585
Line_1_D: <gurobi.LinExpr: -0.5607822357546095 pg[PV_1] + -0.6089602208400834 pg[PV_2]
+ -0.398309791005728 pg[PV_3] + -0.4219560834463752 pg[PV_4] + -0.5066725368514862
pg[PV_5] + -0.5066725368514862 pg[PV_6] + -0.5066725368514856 pg[PV_7] +
-0.5066725368514856 pg[PV_8] + -0.5988298865678321 pg[PV_9] + -0.5607822357546095
 pg[Slack_10]> >= -

In [25]:
ss.Mvsg['PV_1'].X

0.0

In [26]:
ss.Mvsg['PV_8'].X

0.0

In [27]:
ss.Mvsg['PV_6'].X

0.0

In [28]:
ss.Mvsg['PV_9'].X

0.0